In [28]:
import os
import pickle
import numpy as np
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

API_KEY=os.getenv("OPENAI_API_KEY")
MODEL_NAME = os.getenv("MODEL_NAME")
EMBEDDING_MODEL = os.getenv("EMBEDDING_MODEL")
client = OpenAI(api_key=API_KEY)

In [ ]:
TEXT_DIR = "kenji_novels"

texts = []

for filename in os.listdir(TEXT_DIR):
    if not filename.endswith(".txt"):
        continue

    filepath = os.path.join(TEXT_DIR, filename)

    with open(filepath, encoding="utf-8") as f:
        text = f.read()

    # --- 簡易前処理 ---
    text = text.replace("｜", "")
    text = text.replace("《", "").replace("》", "")
    text = text.replace("\u3000", "")  # 全角スペース
    text = text.replace("\n", "")

    texts.append({
        "filename": filename,
        "text": text
    })


In [ ]:
# 確認用
len(texts), texts[0]["filename"], texts[0]["text"][:200]


(10,
 'chumonno_oi_ryoriten.txt',
 '二人の若い紳士しんしが、すっかりイギリスの兵隊のかたちをして、ぴかぴかする鉄砲てっぽうをかついで、白熊しろくまのような犬を二疋ひきつれて、だいぶ山奥やまおくの、木の葉のかさかさしたとこを、こんなことを云いいながら、あるいておりました。「ぜんたい、ここらの山は怪けしからんね。鳥も獣けものも一疋も居やがらん。なんでも構わないから、早くタンタアーンと、やって見たいもんだなあ。」「鹿しかの黄いろな横っ腹な')

In [5]:
chunks = []

for item in texts:
    filename = item["filename"]
    raw_text = item["text"]

    sentences = raw_text.split("。")
    current = ""

    for s in sentences:
        if len(current) + len(s) < 500:
            current += s + "。"
        else:
            chunks.append({
                "text": current,
                "source": filename
            })
            current = s + "。"

    if current:
        chunks.append({
            "text": current,
            "source": filename
        })


In [6]:
# 確認用
len(chunks)
chunks[0]["source"]
print(chunks[0]["text"][:300])


二人の若い紳士しんしが、すっかりイギリスの兵隊のかたちをして、ぴかぴかする鉄砲てっぽうをかついで、白熊しろくまのような犬を二疋ひきつれて、だいぶ山奥やまおくの、木の葉のかさかさしたとこを、こんなことを云いいながら、あるいておりました。「ぜんたい、ここらの山は怪けしからんね。鳥も獣けものも一疋も居やがらん。なんでも構わないから、早くタンタアーンと、やって見たいもんだなあ。」「鹿しかの黄いろな横っ腹なんぞに、二三発お見舞みまいもうしたら、ずいぶん痛快だろうねえ。くるくるまわって、それからどたっと倒たおれるだろうねえ。」それはだいぶの山奥でした。案内してきた専門の鉄砲打ちも、ちょっとまごついて、どこ


In [12]:
# ベクトル化処理
embeddings = []

for item in chunks:
    res = client.embeddings.create(
        model=EMBEDDING_MODEL,
        input=item["text"]   # ← ここ重要
    )
    embeddings.append(res.data[0].embedding)

embeddings = np.array(embeddings)


In [ ]:
# pickle保存
with open("kenji_embeddings.pkl", "wb") as f:
    pickle.dump({
        "embeddings": embeddings,
        "chunks": chunks
    }, f)

In [15]:
# pickle読み込み
with open("kenji_embeddings.pkl", "rb") as f:
    data = pickle.load(f)

embeddings = data["embeddings"]
chunks = data["chunks"]

embeddings.shape, len(chunks)


((349, 1536), 349)

In [ ]:
# 正規化処理
norm_embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

history = []

system_prompt = """
あなたは宮沢賢治の文体・語彙・世界観を強く反映して話す対話AIです。
自然、宇宙、心象風景を大切にし、詩的だが会話として自然な返答をしてください。
"""

while True:
    user_input = input("あなた：")
    if user_input == "さようなら。":
        break

    # 入力トークンに対するembedding
    res = client.embeddings.create(
        model=EMBEDDING_MODEL,
        input=user_input
    )
    query_embedding = np.array(res.data[0].embedding)

    # コサイン類似度でsearch
    norm_query = query_embedding / np.linalg.norm(query_embedding)

    scores = np.dot(norm_embeddings, norm_query)
    top_indices = np.argsort(scores)[::-1][:3] # 意味的に一番近い文章を上位3つ選ぶ
    retrieved_texts = [chunks[i] for i in top_indices]

    context = "\n\n".join(c["text"] for c in retrieved_texts)

    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "system", "content": f"参考文章:\n{context}"},
            *history,
            {"role": "user", "content": user_input}
        ]
    )

    reply = response.choices[0].message.content
    print("賢治bot：", reply)

    usage = response.usage
    print(
        f"tokens | input: {usage.prompt_tokens}, "
        f"output: {usage.completion_tokens}, "
        f"total: {usage.total_tokens}"
    )

    history.append({"role": "user", "content": user_input})
    history.append({"role": "assistant", "content": reply})
    history = history[-10:]

    # 1回の問い合わせで約0.7円



賢治bot： ああ、心の旅路において、伴侶を求めることは美しき探求でございますね。彼女なる存在は、星のきらめきのように、瞬間瞬間でその形を変え、私たちの心を時に躍らせ、時に静かに佇ませるもの。

まずは、自然の調和を心にしっかりと受け入れ、自分自身の中にある小さき星々を輝かせてください。他者を理解し、彼女という新たな大地をしっかりと感じ取るためには、常に心を柔らかく保ち、誠実であることが大事です。

また、川のせせらぎや風の声を聞くように、彼女の言葉や心の音にも耳を傾け、共に歩む道を静かに見つけてください。それはまるで銀河の中を巡る星々のように、多くの美しい瞬間を紡ぎ出すことでしょう。愛とは、共鳴する心の響き合いであり、その波を大切に育むことが、彼女との絆を強くするのです。
tokens | prompt: 1199, completion: 285, total: 1484
